In [1]:
import pandas as pd
import numpy as np
df_temp = pd.read_csv('resources/results/trace_temp.csv')
df_temp.loc[:, ['duration', 'realtime', '%cpu',	'peak_rss',	'peak_vmem', 'rchar', 'wchar']] = 0
df_base = df_temp[~df_temp['name'].str.contains('pearson_corr')]
df_base_dataset = df_base[df_base['name'].str.contains('ridge__op')]
df_base = df_base[~df_base['name'].str.contains('ridge__op')]
df_pearson = df_temp[df_temp['name'].str.contains('pearson_corr')]

In [2]:
trace = pd.read_csv('resources/results/trace.csv', index_col=0)

In [3]:
for row in df_pearson.iterrows():
    print(row[1]['name'])
df_pearson

run_benchmark:run_wf:bench:runEachWf:pearson_corr:processWf:pearson_corr_process (ridge__op.pearson_corr)
run_benchmark:run_wf:bench:runEachWf:pearson_corr:publishFilesSimpleWf:publishFilesProc (ridge__op.pearson_corr)
run_benchmark:run_wf:bench:runEachWf:pearson_corr:publishStatesSimpleWf:publishStatesProc (ridge__op.pearson_corr)
run_benchmark:run_wf:bench:runEachWf:ws_distance:processWf:ws_distance_process (ridge__op.pearson_corr.ws_distance)
run_benchmark:run_wf:bench:score_uns:processWf:score_uns_process (ridge__op.pearson_corr.ws_distance)
run_benchmark:run_wf:bench:runEachWf:regression_1:processWf:regression_1_process (ridge__op.pearson_corr.regression_1)
run_benchmark:run_wf:bench:score_uns:processWf:score_uns_process (ridge__op.pearson_corr.regression_1)
run_benchmark:run_wf:bench:runEachWf:regression_2:processWf:regression_2_process (ridge__op.pearson_corr.regression_2)
run_benchmark:run_wf:bench:score_uns:processWf:score_uns_process (ridge__op.pearson_corr.regression_2)


,task_id,hash,native_id,name,status,exit,submit,duration,realtime,%cpu,peak_rss,peak_vmem,rchar,wchar
1,3,9f/b7088b,2927660,run_benchmark:run_wf:bench:runEachWf:pearson_c...,COMPLETED,0,2025-02-26 19:49:51.948,0,0,0,0,0,0,0
2,39,10/776a87,2950779,run_benchmark:run_wf:bench:runEachWf:pearson_c...,COMPLETED,0,2025-02-26 19:54:12.183,0,0,0,0,0,0,0
3,40,27/0283c3,2950787,run_benchmark:run_wf:bench:runEachWf:pearson_c...,COMPLETED,0,2025-02-26 19:54:12.199,0,0,0,0,0,0,0
4,41,8e/eb6a9c,2950801,run_benchmark:run_wf:bench:runEachWf:ws_distan...,COMPLETED,0,2025-02-26 19:54:12.215,0,0,0,0,0,0,0
5,44,4b/e6e9f8,2951901,run_benchmark:run_wf:bench:score_uns:processWf...,COMPLETED,0,2025-02-26 19:54:15.802,0,0,0,0,0,0,0
6,42,7d/955116,2951605,run_benchmark:run_wf:bench:runEachWf:regressio...,COMPLETED,0,2025-02-26 19:54:14.423,0,0,0,0,0,0,0
7,52,82/0381af,2994528,run_benchmark:run_wf:bench:score_uns:processWf...,COMPLETED,0,2025-02-26 20:10:35.623,0,0,0,0,0,0,0
8,43,1d/164a5b,2951611,run_benchmark:run_wf:bench:runEachWf:regressio...,COMPLETED,0,2025-02-26 19:54:14.445,0,0,0,0,0,0,0
9,53,43/b3d6a7,2995112,run_benchmark:run_wf:bench:score_uns:processWf...,COMPLETED,0,2025-02-26 20:10:54.081,0,0,0,0,0,0,0


In [4]:
methods = trace.index.tolist()
lambda_duration = lambda method, dataset: trace.loc[method, 'Duration (hour)'] if dataset == 'op' else 0 
lambda_memory = lambda method, dataset: trace.loc[method, 'Peak memory (GB)'] if dataset == 'op' else 0
def extend_trace(df, method, datasets):
    df_temp = df.copy()
    for dataset in datasets:
        new_row = df_base_dataset.iloc[0].copy()  # Keep all columns from df_base
        df_temp = pd.concat([df_temp, pd.DataFrame([new_row])], ignore_index=True)
        for _, row in df_pearson.iterrows():
            
            new_row = row.copy()  # Keep all columns from df_base
            new_row['name'] = new_row['name'].replace('pearson_corr', method).replace('op', dataset)
            if f'bench:runEachWf:{method}:processWf' in new_row['name']:
                new_row['duration'] = lambda_duration(method, dataset)

                new_row['peak_vmem'] = lambda_memory(method, dataset)
            
            df_temp = pd.concat([df_temp, pd.DataFrame([new_row])], ignore_index=True)
    
    return df_temp

df_all = df_base.copy()
for method in methods:
    if method in ['portia', 'grnboost2', 'scenic', 'ppcor', 'pearson_corr', 'positive_control', 'negative_control']:
        df_all = extend_trace(df_all, method, datasets=['op', 'norman','adamson','replogle','nakatake'])
    elif method in ['scglue', 'scenicplus', 'figr', 'celloracle', 'granie']:
        df_all = extend_trace(df_all, method, datasets=['op'])
    elif method == 'scprint':
        df_all = extend_trace(df_all, method, datasets=['norman', 'adamson', 'replogle'])
    else:
        raise ValueError(f'Unknown method: {method}')

In [5]:
def format_duration(hours):
    total_seconds = int(hours * 3600)  # Convert hours to seconds
    h, remainder = divmod(total_seconds, 3600)  # Get hours and remaining seconds
    m, s = divmod(remainder, 60)  # Get minutes and remaining seconds
    
    # Construct the formatted string dynamically to avoid unnecessary parts
    formatted = []
    if h > 0:
        formatted.append(f"{h}h")
    if m > 0 or h > 0:  # Include minutes if there are hours, even if minutes are zero
        formatted.append(f"{m}m")
    formatted.append(f"{s}s")
    
    return " ".join(formatted)

df_all['duration'] = df_all['duration'].apply(format_duration)
df_all

,task_id,hash,native_id,name,status,exit,submit,duration,realtime,%cpu,peak_rss,peak_vmem,rchar,wchar
0,58,2b/d7f06e,2996939,run_benchmark:publishStatesSimpleWf:publishSta...,COMPLETED,0,2025-02-26 20:11:36.825,0s,0,0,0,0,0,0
1,57,b6/cf38cd,2996946,run_benchmark:publishFilesSimpleWf:publishFile...,COMPLETED,0,2025-02-26 20:11:36.838,0s,0,0,0,0,0,0
2,1,e9/05f927,2920389,run_benchmark:run_wf:metadata:dataset_uns:proc...,COMPLETED,0,2025-02-26 19:48:10.329,0s,0,0,0,0,0,0
3,3,9f/b7088b,2927660,run_benchmark:run_wf:bench:runEachWf:portia:pr...,COMPLETED,0,2025-02-26 19:49:51.948,49h 49m 20s,0,0,0,55.68523,0,0
4,39,10/776a87,2950779,run_benchmark:run_wf:bench:runEachWf:portia:pu...,COMPLETED,0,2025-02-26 19:54:12.183,0s,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,44,4b/e6e9f8,2951901,run_benchmark:run_wf:bench:score_uns:processWf...,COMPLETED,0,2025-02-26 19:54:15.802,0s,0,0,0,0,0,0
398,42,7d/955116,2951605,run_benchmark:run_wf:bench:runEachWf:regressio...,COMPLETED,0,2025-02-26 19:54:14.423,0s,0,0,0,0,0,0
399,52,82/0381af,2994528,run_benchmark:run_wf:bench:score_uns:processWf...,COMPLETED,0,2025-02-26 20:10:35.623,0s,0,0,0,0,0,0
400,43,1d/164a5b,2951611,run_benchmark:run_wf:bench:runEachWf:regressio...,COMPLETED,0,2025-02-26 19:54:14.445,0s,0,0,0,0,0,0


In [6]:
def format_memory(mem):
    
    
    return f"{mem:.2f} GB"

df_all['peak_vmem'] = df_all['peak_vmem'].apply(format_memory)
df_all

,task_id,hash,native_id,name,status,exit,submit,duration,realtime,%cpu,peak_rss,peak_vmem,rchar,wchar
0,58,2b/d7f06e,2996939,run_benchmark:publishStatesSimpleWf:publishSta...,COMPLETED,0,2025-02-26 20:11:36.825,0s,0,0,0,0.00 GB,0,0
1,57,b6/cf38cd,2996946,run_benchmark:publishFilesSimpleWf:publishFile...,COMPLETED,0,2025-02-26 20:11:36.838,0s,0,0,0,0.00 GB,0,0
2,1,e9/05f927,2920389,run_benchmark:run_wf:metadata:dataset_uns:proc...,COMPLETED,0,2025-02-26 19:48:10.329,0s,0,0,0,0.00 GB,0,0
3,3,9f/b7088b,2927660,run_benchmark:run_wf:bench:runEachWf:portia:pr...,COMPLETED,0,2025-02-26 19:49:51.948,49h 49m 20s,0,0,0,55.69 GB,0,0
4,39,10/776a87,2950779,run_benchmark:run_wf:bench:runEachWf:portia:pu...,COMPLETED,0,2025-02-26 19:54:12.183,0s,0,0,0,0.00 GB,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,44,4b/e6e9f8,2951901,run_benchmark:run_wf:bench:score_uns:processWf...,COMPLETED,0,2025-02-26 19:54:15.802,0s,0,0,0,0.00 GB,0,0
398,42,7d/955116,2951605,run_benchmark:run_wf:bench:runEachWf:regressio...,COMPLETED,0,2025-02-26 19:54:14.423,0s,0,0,0,0.00 GB,0,0
399,52,82/0381af,2994528,run_benchmark:run_wf:bench:score_uns:processWf...,COMPLETED,0,2025-02-26 20:10:35.623,0s,0,0,0,0.00 GB,0,0
400,43,1d/164a5b,2951611,run_benchmark:run_wf:bench:runEachWf:regressio...,COMPLETED,0,2025-02-26 19:54:14.445,0s,0,0,0,0.00 GB,0,0


In [7]:
df_all['task_id'] = np.arange(1, len(df_all)+1)
df_all.to_csv('resources/results/all_main_fixed/trace.txt', sep='\t', index=False)